In [1]:
import pandas as pd
import numpy as np
from sklearn.covariance import ledoit_wolf

# Constantes

In [2]:
# Número de acciones en el portafolio
PORTFOLIO_SIZE = 300
# Número de portafolios a generar
NUM_PORTFOLIOS = 50
# Rutas de S3
S3_REFINED_URI = 's3://proyecto-integrador-20212-pregrado/datasets/refined/'
# Número de días usados para calcular la rentabilidad y desviación estándar
LOOKBACK_PERIOD = 252

# Cargar portafolios y matrices de covarianzas de S3

In [4]:
# Cargar los portafolios aleatorios
portfolios = [i for i in range(NUM_PORTFOLIOS)]
for i in portfolios:
    print(i, end=', ')
    portfolios[i] = pd.read_parquet(S3_REFINED_URI+f'portfolio_{i}_returns.parquet')

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 

In [5]:
# Cargar las matrices de varianzas y covarianzas de S3
cov_matrices = [i for i in range(NUM_PORTFOLIOS)]
cov_matrices_lw = [i for i in range(NUM_PORTFOLIOS)]

for i in cov_matrices:
    print(i, end=', ')
    cov_matrices[i] = pd.read_parquet(S3_REFINED_URI+f'portfolio_{i}_cov.parquet')
    cov_matrices_lw[i] = pd.read_parquet(S3_REFINED_URI+f'portfolio_{i}_cov_lw.parquet')

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 

# Funciones

In [221]:
# Función para sacar un portafolio de PORTFOLIO_SIZE acciones escogidas aleatoriamente
def select_random_stocks(stock_names, n_stocks=PORTFOLIO_SIZE):
    return np.random.choice(stock_names, size=PORTFOLIO_SIZE, replace=False)

# Función para calcular el retorno promedio del portafolio durante los últimos <window> períodos
def portfolio_mean_returns(df_returns, ws, window=LOOKBACK_PERIOD):
    return pd.Series(np.dot(df_returns, ws), index=df_returns.index).rolling(window).mean().iloc[window:]


# Función para calcular la desviación estándar del portafolio
def portfolio_std(df_returns, ws, cov, window=LOOKBACK_PERIOD):
    # Retornar la desviación estándar diaria del portafolio
    return (ws.T.dot(cov).dot(ws)/LOOKBACK_PERIOD)**.5

# Función para calcular los retornos promedio para una lista de portafolios
def calculate_portfolio_returns(portfolios, ws):
    portfolio_returns_matrix = pd.DataFrame()

    for i, portfolio in enumerate(portfolios):
        portfolio_name = f'portfolio_{i}'
        portfolio_returns_matrix[portfolio_name] = portfolio_mean_returns(portfolio, ws[i])
        
    return portfolio_returns_matrix

# Función para calcular las desviaciones estándar de una lista de portafolios
def calculate_portfolio_std(portfolios, ws, cov_matrices):
    portfolio_stds_matrix = pd.Series(index=[f'portfolio_{i}' for i in range(NUM_PORTFOLIOS)],
                                            dtype=np.float64)

    for i, portfolio in enumerate(portfolios):
        portfolio_stds_matrix[i] = portfolio_std(portfolio, ws[i], cov_matrices[i])
        
    return portfolio_stds_matrix

# Función para calcular los retornos diarios promedio ajustados por riesgo
def calculate_portfolio_risk_adjusted_returns(portfolio_returns_matrix, portfolio_stds_matrix):
    return portfolio_returns_matrix/portfolio_stds_matrix

# Función para calcular los pesos del portafolio de mínima varianza para una matriz de retornos
def calculate_minimum_variance_weights(portfolios, cov_matrices, moore_penrose=False):
    ws_minimum_variance = [0 for i in portfolios]
    ones = np.ones(PORTFOLIO_SIZE)
    for i, portfolio in enumerate(portfolios):
        if moore_penrose:
            cov_inv = np.linalg.pinv(cov_matrices[i])
        else:
            cov_inv = np.linalg.inv(cov_matrices[i])
            
        numerator = ones @ cov_inv
        denominator = ones.T @ cov_inv @ ones
        
        ws_minimum_variance[i] = numerator/denominator
        
    return ws_minimum_variance

# Portafolio con pesos iguales

In [222]:
# Calcular el peso de cada activo para el escenario de igualdad de pesos
ws_constant = np.array([np.ones(PORTFOLIO_SIZE)*(1/PORTFOLIO_SIZE) for i in portfolios])
print(f'Cada acción tendrá un peso de {(ws_constant[0][0]*100).round(2)}%')

Cada acción tendrá un peso de 0.33%


In [223]:
# Calcular el retorno promedio de los últimos <LOOKBACK_PERIOD> días para cada fecha para cada portafolio
portfolio_returns_equal_weights = calculate_portfolio_returns(portfolios, ws_constant)

In [224]:
#portfolio_returns_equal_weights

In [225]:
# Calcular las desviaciones estándar de cada portafolio
portfolio_stds_equal_weights = calculate_portfolio_std(portfolios, ws_constant, cov_matrices)

In [226]:
#portfolio_stds_equal_weights

In [227]:
# Calcular los retornos diarios promedio ajustados por riesgo
portfolio_risk_adjusted_returns_equal_weights = \
    calculate_portfolio_risk_adjusted_returns(portfolio_returns_equal_weights, portfolio_stds_equal_weights)

In [228]:
portfolio_risk_adjusted_returns_equal_weights

,portfolio_0,portfolio_1,portfolio_2,portfolio_3,portfolio_4,portfolio_5,portfolio_6,portfolio_7,portfolio_8,portfolio_9,...,portfolio_40,portfolio_41,portfolio_42,portfolio_43,portfolio_44,portfolio_45,portfolio_46,portfolio_47,portfolio_48,portfolio_49
date,,,,,,,,,,,,,,,,,,,,,
2014-12-13,0.430913,0.429936,0.364174,0.366605,0.368444,0.386220,0.360768,0.367419,0.351052,0.409374,...,0.396453,0.351496,0.433328,0.493895,0.372460,0.343323,0.343607,0.405317,0.368370,0.320877
2014-12-15,0.409390,0.408854,0.342665,0.346364,0.345428,0.363176,0.337305,0.347362,0.331137,0.388518,...,0.377020,0.329060,0.412706,0.467703,0.349000,0.319995,0.316624,0.383893,0.348601,0.296942
2014-12-16,0.387304,0.390221,0.321515,0.322487,0.330046,0.342987,0.320233,0.330801,0.313770,0.369573,...,0.357057,0.310652,0.394173,0.445931,0.331683,0.301737,0.298690,0.362618,0.330764,0.277947
2014-12-17,0.473018,0.464889,0.400701,0.398407,0.407910,0.425939,0.399068,0.410098,0.393279,0.450204,...,0.432942,0.386582,0.468866,0.533712,0.408668,0.381733,0.375736,0.440301,0.408628,0.361216
2014-12-18,0.574502,0.557943,0.492229,0.488790,0.493802,0.517975,0.489737,0.500040,0.483484,0.540441,...,0.520204,0.474077,0.550514,0.636627,0.497453,0.467870,0.465787,0.526365,0.496467,0.451791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-19,1.068958,2.102415,1.999484,2.100543,2.079684,2.048619,2.079080,2.111807,2.035710,2.042861,...,1.986429,2.088982,2.023425,1.018984,1.986119,2.034433,2.061818,2.026127,2.066688,2.006356
2020-11-20,1.034587,2.074887,1.968721,2.070316,2.052033,2.019484,2.052809,2.085173,2.002866,2.013422,...,1.956654,2.060431,1.994327,0.990508,1.958324,2.007935,2.033970,1.994032,2.040057,1.978987
2020-11-23,1.072871,2.110265,2.004772,2.100160,2.089232,2.054583,2.088341,2.120498,2.040793,2.045789,...,1.997437,2.090202,2.030546,1.024584,1.993531,2.047548,2.071860,2.029449,2.076168,2.016474


In [229]:
# Guardar la matriz de retornos ajustados por riesgo para cada portafolio en S3
portfolio_risk_adjusted_returns_equal_weights.to_parquet(
    S3_REFINED_URI+'matriz_retornos_ajustados_por_riesgo_pesos_iguales.parquet')

# Portafolio de mínima varianza tradicional

El vector de pesos $w$ del portafolio de mínima varianza está dado por
$$
w_{MV} = \frac{\Sigma^{-1}1}{1^T\Sigma^{-1}1}
$$
donde,<br>
$\Sigma^{-1}$ es la matriz inversa de la matriz de covarianzas de orden $n \times n$<br>
$1$ es un vector de 1s de orden $n \times 1$

In [230]:
# Calcular los pesos para el portafolio de mínima varianza
ws_minimum_variance = calculate_minimum_variance_weights(portfolios, cov_matrices)

In [231]:
# Calcular el retorno promedio de los últimos <LOOKBACK_PERIOD> días para cada fecha para cada portafolio
portfolio_returns_minimum_variance = calculate_portfolio_returns(portfolios, ws_minimum_variance)

In [232]:
#portfolio_returns_minimum_variance

In [233]:
# Calcular las desviaciones estándar de cada portafolio
portfolio_stds_minimum_variance = calculate_portfolio_std(portfolios, ws_minimum_variance, cov_matrices)

In [234]:
#portfolio_stds_minimum_variance

In [235]:
# Calcular los retornos diarios promedio ajustados por riesgo
portfolio_risk_adjusted_returns_minimum_variance = \
    calculate_portfolio_risk_adjusted_returns(portfolio_returns_minimum_variance, portfolio_stds_minimum_variance)

In [236]:
portfolio_risk_adjusted_returns_minimum_variance

,portfolio_0,portfolio_1,portfolio_2,portfolio_3,portfolio_4,portfolio_5,portfolio_6,portfolio_7,portfolio_8,portfolio_9,...,portfolio_40,portfolio_41,portfolio_42,portfolio_43,portfolio_44,portfolio_45,portfolio_46,portfolio_47,portfolio_48,portfolio_49
date,,,,,,,,,,,,,,,,,,,,,
2014-12-13,0.894906,1.138584,1.019465,1.393081,1.230530,1.368738,1.046779,1.423632,1.237577,1.025844,...,0.754162,0.980963,1.256903,1.485507,1.271698,1.167690,0.637763,1.207524,1.609587,0.765683
2014-12-15,0.855066,1.041695,0.924681,1.299429,1.158183,1.275422,0.977054,1.360116,1.197987,0.930027,...,0.675912,0.916196,1.180463,1.361624,1.172116,1.105202,0.496776,1.169463,1.526732,0.706841
2014-12-16,0.975212,1.125981,1.041144,1.421999,1.225654,1.357765,1.107216,1.465100,1.243561,0.997434,...,0.729252,1.022726,1.227655,1.401465,1.206625,1.195841,0.588873,1.315036,1.624386,0.744218
2014-12-17,0.917422,1.074692,0.955468,1.316868,1.205034,1.305691,1.049868,1.418537,1.223829,0.922063,...,0.668365,0.932416,1.076170,1.335673,1.160531,1.166979,0.509940,1.273726,1.556476,0.693748
2014-12-18,0.958717,1.141633,1.105916,1.413014,1.252100,1.376950,1.208385,1.515719,1.244639,1.020181,...,0.813776,1.070147,1.214203,1.455025,1.250185,1.264570,0.663517,1.297210,1.706507,0.776396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-19,1.956441,1.732254,1.431624,1.766004,2.253776,1.536946,1.112356,1.724031,1.355366,1.739918,...,1.243333,1.523673,1.469182,1.549600,2.720376,1.259701,1.090915,2.115699,1.628361,1.501647
2020-11-20,1.935192,1.746447,1.415558,1.752719,2.260180,1.553128,1.049777,1.699091,1.386494,1.667397,...,1.132896,1.490032,1.436899,1.508208,2.694488,1.121507,1.055763,2.102437,1.570748,1.489306
2020-11-23,1.942301,1.719130,1.347149,1.798188,2.267683,1.511434,0.992918,1.728454,1.384872,1.653508,...,1.164207,1.573448,1.388309,1.506439,2.706616,1.113161,1.059530,2.110412,1.575658,1.457386


In [237]:
# Guardar la matriz de retornos ajustados por riesgo para cada portafolio en S3
portfolio_risk_adjusted_returns_equal_weights.to_parquet(
    S3_REFINED_URI+'matriz_retornos_ajustados_por_riesgo_varianza_mínima.parquet')

# Portafolio con shrinkage de Ledoit & Wolf

In [238]:
# Calcular los pesos para el portafolio de mínima varianza con Ledoit & Wolf
ws_minimum_variance_lw = calculate_minimum_variance_weights(portfolios, cov_matrices_lw, moore_penrose=True)

In [239]:
# Calcular el retorno promedio de los últimos <LOOKBACK_PERIOD> días para cada fecha para cada portafolio
portfolio_returns_minimum_variance_lw = calculate_portfolio_returns(portfolios, ws_minimum_variance_lw)

In [240]:
#portfolio_returns_minimum_variance

In [241]:
# Calcular las desviaciones estándar de cada portafolio
portfolio_stds_minimum_variance_lw = calculate_portfolio_std(portfolios, ws_minimum_variance_lw, cov_matrices_lw)

In [242]:
#portfolio_stds_minimum_variance

In [243]:
# Calcular los retornos diarios promedio ajustados por riesgo
portfolio_risk_adjusted_returns_minimum_variance_lw = \
    calculate_portfolio_risk_adjusted_returns(portfolio_returns_minimum_variance_lw, portfolio_stds_minimum_variance_lw)

In [244]:
portfolio_risk_adjusted_returns_minimum_variance_lw

,portfolio_0,portfolio_1,portfolio_2,portfolio_3,portfolio_4,portfolio_5,portfolio_6,portfolio_7,portfolio_8,portfolio_9,...,portfolio_40,portfolio_41,portfolio_42,portfolio_43,portfolio_44,portfolio_45,portfolio_46,portfolio_47,portfolio_48,portfolio_49
date,,,,,,,,,,,,,,,,,,,,,
2014-12-13,0.849904,0.950433,0.819817,0.811604,0.809915,0.859592,0.797824,0.821615,0.794188,0.910593,...,0.883715,0.783500,0.966582,1.025846,0.828414,0.770180,0.763736,0.898235,0.818214,0.720603
2014-12-15,0.808347,0.903827,0.771396,0.766794,0.759322,0.808305,0.745937,0.776764,0.749135,0.864202,...,0.840398,0.733490,0.920583,0.955734,0.776235,0.717848,0.703759,0.850757,0.774302,0.666850
2014-12-16,0.904583,0.862638,0.723784,0.713933,0.725509,0.763370,0.708183,0.739732,0.709845,0.822061,...,0.795900,0.692458,0.879243,0.987241,0.737719,0.676889,0.663898,0.803608,0.734683,0.624194
2014-12-17,0.865008,1.027701,0.902044,0.882009,0.896670,0.947994,0.882524,0.917054,0.889719,1.001413,...,0.965050,0.861709,1.045852,0.929712,0.908947,0.856345,0.835149,0.975763,0.907633,0.811192
2014-12-18,0.951111,1.233410,1.108089,1.082101,1.085477,1.152834,1.083034,1.118181,1.093791,1.202132,...,1.159564,1.056738,1.227977,1.041725,1.106418,1.049577,1.035304,1.166492,1.102739,1.014599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-19,1.140494,4.647678,4.501174,4.650263,4.571566,4.559523,4.597806,4.722385,4.605408,4.544047,...,4.427858,4.656434,4.513453,1.258907,4.417464,4.563858,4.582802,4.490159,4.590470,4.505731
2020-11-20,1.106673,4.586822,4.431922,4.583343,4.510784,4.494678,4.539709,4.662826,4.531104,4.478564,...,4.361489,4.592793,4.448546,1.226584,4.355644,4.504417,4.520905,4.419033,4.531317,4.444268
2020-11-23,1.111863,4.665032,4.513078,4.649414,4.592556,4.572796,4.618287,4.741819,4.616908,4.550558,...,4.452396,4.659155,4.529336,1.217435,4.433951,4.593281,4.605123,4.497520,4.611526,4.528454


In [245]:
# Guardar la matriz de retornos ajustados por riesgo para cada portafolio en S3
portfolio_risk_adjusted_returns_equal_weights.to_parquet(
    S3_REFINED_URI+'matriz_retornos_ajustados_por_riesgo_varianza_mínima_lw.parquet')

In [246]:
# Retornos ajustados por riesgo promedio con Ledoit & Wolf vs mínima varianza
risk_adjusted_lw_minus_min_var = \
    portfolio_risk_adjusted_returns_minimum_variance_lw.mean()-portfolio_risk_adjusted_returns_minimum_variance.mean()

In [247]:
# Retornos ajustados por riesgo promedio con mínima varianza vs pesos iguales
risk_adjusted_min_var_minus_equal_weights = \
    portfolio_risk_adjusted_returns_minimum_variance.mean()-portfolio_risk_adjusted_returns_equal_weights.mean()

In [248]:
# Retornos ajustados por riesgo promedio con Ledoit & Wolf vs pesos iguales
risk_adjusted_lw_minus_equal_weights = \
    portfolio_risk_adjusted_returns_minimum_variance_lw.mean()-portfolio_risk_adjusted_returns_equal_weights.mean()

In [249]:
# Juntar los resultados en un DataFrame
risk_adjusted_comparisons = pd.DataFrame()
risk_adjusted_comparisons['Mínima varianza - Pesos iguales'] = risk_adjusted_min_var_minus_equal_weights
risk_adjusted_comparisons['Mínima varianza con LW - Pesos iguales'] = risk_adjusted_lw_minus_equal_weights
risk_adjusted_comparisons['Mínima varianza con LW - Mínima Varianza'] = risk_adjusted_lw_minus_min_var

risk_adjusted_comparisons

,Mínima varianza - Pesos iguales,Mínima varianza con LW - Pesos iguales,Mínima varianza con LW - Mínima Varianza
portfolio_0,0.332730,0.195546,-0.137185
portfolio_1,0.057042,0.541814,0.484772
portfolio_2,0.286261,0.488819,0.202559
portfolio_3,0.050811,0.456099,0.405287
portfolio_4,0.379560,0.499033,0.119473
portfolio_5,0.167878,0.484054,0.316176
portfolio_6,-0.002091,0.478464,0.480555
portfolio_7,0.090421,0.514158,0.423737
portfolio_8,0.195464,0.484951,0.289488
portfolio_9,0.471966,0.506866,0.034900


In [250]:
# Escribir comparaciones a S3
risk_adjusted_comparisons.to_parquet(f'{S3_REFINED_URI}comparaciones_retorno_ajustado_por_riesgo.parquet')